# ML Monitoring Fundamentals

## Setup

Required to run this notebook
- [Free WhyLabs Account](https://whylabs.ai/free)

Reference to whylogs:
- whylogs [GitHub](https://github.com/whylabs/whylogs/)





In [ ]:
# Install whylogs
!pip install 'whylogs[viz]'


# 1. Data Drift, Model Drift, Performance


In [ ]:
import whylogs as why
import numpy as np
import pandas as pd
import datetime
import os

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

pd.set_option("display.max_columns", None)

## Train a Machine Learning Model

In [ ]:
df_iris = load_iris(as_frame=True)

print(list(df_iris.target_names))
print(list(df_iris.feature_names))

In [ ]:
# Train baseline Model
# KNN Model
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

# Create feature and target data varaible
X, y = df_iris.data, df_iris.target

#create train & test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=6,
                                                    stratify=y)
# Train model
knn.fit(X_train, y_train)

# Predict the labels on test data sset
y_pred = knn.predict(X_test)

# Print model accuracy
knn.score(X_test, y_test)


## Import batches of data

In [ ]:
 # Import data batches
url = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_1_no_drift.csv'
batch_data_1 = pd.read_csv(url)

url2 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_2_no_drift.csv'
batch_data_2 = pd.read_csv(url2)

url3 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_3_no_drift.csv'
batch_data_3 = pd.read_csv(url3)

url4 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_4_drift_0s.csv'
batch_data_4 = pd.read_csv(url4)

url5 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_5_drift.csv'
batch_data_5 = pd.read_csv(url5)

url6 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_6_drift.csv'
batch_data_6 = pd.read_csv(url6)

url7 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_7_no_drift.csv'
batch_data_7 = pd.read_csv(url7)

# iris feature names
feature_names = ['sepal length (cm)', 'sepal width (cm)','petal length (cm)','petal width (cm)']

# separate targets
X_batch_1 = batch_data_1[feature_names]
X_batch_2 = batch_data_2[feature_names]
X_batch_3 = batch_data_3[feature_names]
X_batch_4 = batch_data_4[feature_names]
X_batch_5 = batch_data_5[feature_names]
X_batch_6 = batch_data_6[feature_names]
X_batch_7 = batch_data_7[feature_names]

y_batch_1 = batch_data_1['target']
y_batch_2 = batch_data_2['target']
y_batch_3 = batch_data_3['target']
y_batch_4 = batch_data_4['target']
y_batch_5 = batch_data_5['target']
y_batch_6 = batch_data_6['target']
y_batch_7 = batch_data_7['target']



dfs = [X_batch_1, X_batch_4, X_batch_5, X_batch_6, X_batch_2, X_batch_3, X_batch_7]

df_target = [y_batch_1, y_batch_4, y_batch_5, y_batch_6, y_batch_2, y_batch_3, y_batch_7]


In [ ]:
X_batch_1

In [ ]:
dfs[0].head()

## Create a log with whylogs

whylogs is an open source library for logging any kind of data. With whylogs, users are able to generate summaries of their datasets (called whylogs profiles) which they can use to:

- Track changes in their dataset
- Create data constraints to know whether their data looks the way it should
- Quickly visualize key summary statistics about their datasets


![](https://user-images.githubusercontent.com/7946482/171062942-01c420f2-7768-4b7c-88b5-e3f291e1b7d8.png)

profiles generated with whylogs are:
- Efficient
- Customizable
- Mergeable


In [ ]:
# create profile
profile1 = why.log(X_batch_1)

profile_view1 = profile1.view()
profile_view1.to_pandas()

## Writing profiles to WhyLabs

We're going start with an example of using profiles with the WhyLabs Observatory.

We'll explore using whylogs for data validation & drift visualization after this!


## Get WhyLabs access tokens [expand]





Before integrate our data into WhyLabs we need three things:
- WhyLabs API Key
- WhyLabs Org-ID
- Project-ID


The easiest way to get the API token & ord-id:

`Menu -> Settings -> Access Tokens`

![](https://github.com/sagecodes/workshop-images/blob/master/access_token_org.png?raw=true)


## Sending profiles

In [ ]:
# set authentication & project keys
os.environ["WHYLABS_DEFAULT_ORG_ID"] = 'org-rdPzFz'
os.environ["WHYLABS_API_KEY"] = 'mH7YHZoCXk.FTQJZRgLvSh6uLuit3bDrMoNlGgu9OZubpEdGvq0nzbKCN7SI2INa:org-rdPzFz'
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = 'model-3'

In [ ]:
from whylogs.api.writer.whylabs import WhyLabsWriter

In [ ]:
# Single Profile
writer = WhyLabsWriter()
profile= why.log(X_batch_1)
writer.write(file=profile.view())

Write multiple profiles with different dates to backfill

In [ ]:
# initialize writer
writer = WhyLabsWriter()

# back fill 1 day per batch
for i, df in enumerate(dfs):

    # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
    dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)

    # create profile for each batch of data
    profile = why.log(df).profile()

    # set the dataset timestamp for the profile
    profile.set_dataset_timestamp(dt)
    # write the profile to the WhyLabs platform
    writer.write(file=profile.view())

Reference Profile

In [ ]:
ref_profile = why.log(df_iris.data).profile()
writer = WhyLabsWriter().option(reference_profile_name="iris_training_profile")
writer.write(file=ref_profile.view())

## Logging output

In [ ]:
# Get predictions with model & append to df
pred_dfs = dfs

class_names = ['setosa', 'versicolor', 'virginica']

for i, df in enumerate(pred_dfs):
    y_pred = knn.predict(df)
    y_prob = knn.predict_proba(df)
    pred_scores = []
    pred_classes = []

    for pred in y_pred:
      pred_classes.append(class_names[pred])
    df['cls_output'] = pred_classes
    for prob in y_prob:
      pred_scores.append(max(prob))
    df['prob_output'] = pred_scores
    # print(pred_scores)

In [ ]:
pred_dfs[-1]

In [ ]:
writer = WhyLabsWriter()

for i, df in enumerate(pred_dfs):

    out_df = df[['cls_output', 'prob_output']].copy()
   # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
    dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)
    profile = why.log(out_df).profile()

    # set the dataset timestamp for the profile
    profile.set_dataset_timestamp(dt)
    #write the profile to the WhyLabs platform
    writer.write(file=profile.view())

## Log performance

Instead of just logging outputs, if we have ground truth data we can also monitor performance metrics overtime.


Classification:

Regression:


In [ ]:
pred_dfs[-1]

In [ ]:
# Append ground truth data to dataframe
for i, df in enumerate(pred_dfs):
    df['ground_truth'] = df_target[i]

In [ ]:
pred_dfs[0]

In [ ]:
# Log performance

for i, df in enumerate(pred_dfs):

  results = why.log_classification_metrics(
          df,
          target_column = "ground_truth",
          prediction_column = "cls_output",
          score_column="prob_output"
      )
   # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
  dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)

  profile = results.profile()
  profile.set_dataset_timestamp(dt)

  results.writer("whylabs").write()

# 2. Monitoring for Bias & Fairness with Tracing & Explainability



In [ ]:
# Imports
import whylogs as why
import numpy as np
import pandas as pd
import datetime
import os

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

pd.set_option("display.max_columns", None)

In [ ]:
# Load iris data as dataframe(df)
df_iris = load_iris(as_frame=True)

print(list(df_iris.target_names))
print(list(df_iris.feature_names))

## Train a Machine Learning Model (quickly)

In [ ]:
from sklearn import neighbors
knn = neighbors.KNeighborsClassifier(n_neighbors=5)

X, y = df_iris.data, df_iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=y)
# Train model
knn.fit(X_train, y_train)

# Predict the labels on test data sset
y_pred = knn.predict(X_test)

# Print model accuracy
knn.score(X_test, y_test)

## Import data batches

In [ ]:
 # Import data batches
url = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_8_statefl_1.csv'
batch_data_1 = pd.read_csv(url)

url2 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_9_statefl_1.csv'
batch_data_2 = pd.read_csv(url2)

url3 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_10_statefl_1.csv'
batch_data_3 = pd.read_csv(url3)

url4 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_11_statefl_1.csv'
batch_data_4 = pd.read_csv(url4)

url5 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_12_statefl_1.csv'
batch_data_5 = pd.read_csv(url5)

url6 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_13_statefl_1.csv'
batch_data_6 = pd.read_csv(url6)

url7 = 'https://raw.githubusercontent.com/manifoldailearning/Complete-MLOps-BootCamp/main/ML-Monitoring-WhyLogs/datasets/iris_14_statefl_1.csv'
batch_data_7 = pd.read_csv(url7)

# iris feature names
feature_names = ['sepal length (cm)', 'sepal width (cm)','petal length (cm)','petal width (cm)', 'state']

# separate targets
X_batch_1 = batch_data_1[feature_names]
X_batch_2 = batch_data_2[feature_names]
X_batch_3 = batch_data_3[feature_names]
X_batch_4 = batch_data_4[feature_names]
X_batch_5 = batch_data_5[feature_names]
X_batch_6 = batch_data_6[feature_names]
X_batch_7 = batch_data_7[feature_names]

# We'll save the target values for later!
y_batch_1 = batch_data_1['target']
y_batch_2 = batch_data_2['target']
y_batch_3 = batch_data_3['target']
y_batch_4 = batch_data_4['target']
y_batch_5 = batch_data_5['target']
y_batch_6 = batch_data_6['target']
y_batch_7 = batch_data_7['target']


# create list of our batches
dfs = [X_batch_1, X_batch_4, X_batch_5, X_batch_6, X_batch_2, X_batch_3, X_batch_7]

df_target = [y_batch_1, y_batch_4, y_batch_5, y_batch_6, y_batch_2, y_batch_3, y_batch_7]


In [ ]:
dfs[0].head()


## Creating profiles with whylogs


Profiles generated with whylogs are:

- Secure
- Efficient
- Customizable
- Mergeable

In [ ]:
# create profile
profile1 = why.log(X_batch_1)

profile_view1 = profile1.view()
profile_view1.to_pandas()

In [ ]:
# set authentication & project keys
# os.environ["WHYLABS_DEFAULT_ORG_ID"] = 'ORGID'
# os.environ["WHYLABS_API_KEY"] = 'APIKEY'
os.environ["WHYLABS_DEFAULT_DATASET_ID"] = 'MODELID'

### Create dataframe with model predictions

In [ ]:
# Get predictions with model & append to df
pred_dfs = dfs

class_names = ['setosa', 'versicolor', 'virginica']

for i, df in enumerate(pred_dfs):
    y_pred = knn.predict(df.iloc[:, :4])
    y_prob = knn.predict_proba(df.iloc[:, :4])
    pred_scores = []
    pred_classes = []

    for pred in y_pred:
      pred_classes.append(class_names[pred])
    df['cls_output'] = pred_classes
    for prob in y_prob:
      pred_scores.append(max(prob))
    df['prob_output'] = pred_scores

In [ ]:
pred_dfs[-1]

### Backfilling data in WhyLabs

In [ ]:
from whylogs.core.schema import DatasetSchema
from whylogs.core.segmentation_partition import segment_on_column

In [ ]:
# back fill 1 day per batch
for i, df in enumerate(pred_dfs):
    # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
    dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)

    # create profile for each batch of data
    profile = why.log(df, schema=DatasetSchema(segments=segment_on_column("state")))

    # set the dataset timestamp for the profile
    profile.set_dataset_timestamp(dt)
    # write the profile to the WhyLabs platform
    profile.writer("whylabs").write()

Learn more about segmentation in whylogs
- [Intro to Segmentation with whylogs](https://github.com/whylabs/whylogs/blob/mainline/python/examples/advanced/Segments.ipynb)

In [ ]:
# Create reference profile
ref_profile = why.log(df_iris.data).profile()
writer = WhyLabsWriter().option(reference_profile_name="iris_training_profile")
writer.write(file=ref_profile.view())

###Classification Performance Metrics

In [ ]:
# Append ground truth data to dataframe
for i, df in enumerate(pred_dfs):
    df['ground_truth'] = df_target[i]

In [ ]:
pred_dfs[0]

In [ ]:
from whylogs import log_classification_metrics
# from whylogs.core.schema import DatasetSchema
# from whylogs.core.segmentation_partition import segment_on_column

In [ ]:
for i, df in enumerate(pred_dfs):

  segmented_classification_results = log_classification_metrics(
    df,
    target_column = "ground_truth",
    prediction_column = "cls_output",
    schema = DatasetSchema(segments=segment_on_column("state"))
  )
   # walking backwards. Each dataset has to map to a date to show up as a different batch in WhyLabs
  dt = datetime.datetime.now(tz=datetime.timezone.utc) - datetime.timedelta(days=i)

  # profile = segmented_classification_results.profile()
  segmented_classification_results.set_dataset_timestamp(dt)

  segmented_classification_results.writer("whylabs").write()

## Feature importance

Learn more about SHAP
https://github.com/slundberg/shap

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
explainer = shap.Explainer(knn.predict, X_train)

In [ ]:
shap_values = explainer(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
# Get global featue importance
shap_feature_importance = np.mean(np.abs(shap_values.values), axis=0)

In [ ]:
# Create dict with feature importance
shap_feature_importance_dict = dict(zip(X_train.columns.tolist(), shap_feature_importance.tolist()))
feature_importance_dict = {k: v for k, v in sorted(shap_feature_importance_dict.items(),
                                                   key=lambda item: item[1], reverse=True)}


In [ ]:
print(feature_importance_dict)

In [ ]:
# Write values to WhyLabs
from whylogs.core.feature_weights import FeatureWeights
from whylogs.api.writer.whylabs import WhyLabsWriter

feature_weights = FeatureWeights(shap_feature_importance_dict)
result = feature_weights.writer("whylabs").write()

result

# 3. Open-source data & ML monitoring with whylogs

## Using data drift reports with whylogs in a Python environment

![](https://whylabs.ai/_next/image?url=https%3A%2F%2Fcontent.whylabs.ai%2Fcontent%2Fimages%2F2022%2F06%2FTDSImage3.jpeg&w=3120&q=75)








In [ ]:
# creat profiles of batches

profile_view1 = why.log(X_batch_1).view()
profile_view1 = why.log(X_batch_1).view()
profile_view2 = why.log(X_batch_2).view()
profile_view3 = why.log(X_batch_3).view()
profile_view4 = why.log(batch_data_4).view()
profile_view5 = why.log(batch_data_5).view()
profile_view6 = why.log(batch_data_6).view()
profile_view7 = why.log(X_batch_7).view()
# profile_view8 = why.log(batch_data_8).view()

In [ ]:
# Data Drift with whylogs
from whylogs.viz import NotebookProfileVisualizer

visualization = NotebookProfileVisualizer()
visualization.set_profiles(target_profile_view=profile_view1, reference_profile_view=profile_view2)

In [ ]:
visualization.summary_drift_report()

In [ ]:
visualization.double_histogram(feature_name="petal width (cm)")


In [ ]:
visualization.double_histogram(feature_name="petal length (cm)")


In [ ]:

from whylogs.viz.drift.column_drift_algorithms import calculate_drift_scores

scores = calculate_drift_scores(target_view=profile_view1, reference_view=profile_view2, with_thresholds = True)

scores

In [ ]:
# Compare Another profiles:

from whylogs.viz import NotebookProfileVisualizer

visualization = NotebookProfileVisualizer()
visualization.set_profiles(target_profile_view=profile_view1, reference_profile_view=profile_view3)

In [ ]:
visualization.summary_drift_report()

In [ ]:
visualization.double_histogram(feature_name="petal length (cm)")


In [ ]:
visualization.double_histogram(feature_name="petal width (cm)")


In [ ]:

from whylogs.viz.drift.column_drift_algorithms import calculate_drift_scores

scores = calculate_drift_scores(target_view=profile_view1, reference_view=profile_view6, with_thresholds = True)

scores

Learn more about using data drift reports with whylogs
- [Drift Algorithm Configuration](https://github.com/whylabs/whylogs/blob/mainline/python/examples/advanced/Drift_Algorithm_Configuration.ipynb)



## Data validation with constraints in whylogs


Data quality validation ensures data is structured and falls in the range expected for our data pipelines or applications. When collecting or using data it’s important to verify the quality to avoid unwanted machine learning behavior in production, such as errors or faulty prediction results.

For example, we may want to ensure our data doesn’t contain any empty or negative values before moving it along in the pipeline if our model does not expect those values.

In [ ]:
# Data Quality Validation whylogs

from whylogs.core.constraints import (Constraints,
                                     ConstraintsBuilder,
                                     MetricsSelector,
                                     MetricConstraint)

In [ ]:
# Using Constraints for Data Quality Validation

def validate_features(profile_view, verbose=False):

  builder = ConstraintsBuilder(profile_view)

  # Define a constraint for validating data
  builder.add_constraint(MetricConstraint(
    name="petal length > 0 and < 15",
    condition=lambda x: x.min > 0 and x.max < 15,
    metric_selector=MetricsSelector(metric_name='distribution',
                                    column_name='petal length (cm)')
  ))

  builder.add_constraint(MetricConstraint(
    name="petal width > 0 and < 15",
    condition=lambda x: x.min > 0 and x.max < 15,
    metric_selector=MetricsSelector(metric_name='distribution',
                                    column_name='petal width (cm)')
  ))

  builder.add_constraint(MetricConstraint(
    name="sepal length > 0 and < 15",
    condition=lambda x: x.min > 0 and x.max < 15 ,
    metric_selector=MetricsSelector(metric_name='distribution',
                                    column_name='sepal length (cm)')
  ))

  builder.add_constraint(MetricConstraint(
    name="sepal width > 0 and < 15",
    condition=lambda x: x.min > 0 and x.max < 15,
    metric_selector=MetricsSelector(metric_name='distribution',
                                    column_name='sepal width (cm)')
  ))

  # Build the constraints and return the report
  constraints: Constraints = builder.build()

  if verbose:
    print(constraints.report())

  # return constraints.report()
  return constraints


In [ ]:
const = validate_features(profile_view2, True)

In [ ]:
from whylogs.viz import NotebookProfileVisualizer
visualization = NotebookProfileVisualizer()
visualization.constraints_report(const, cell_height=300)

In [ ]:
# check all constraints for passing:
constraints_valid = const.validate()
print(constraints_valid)

In [ ]:
const = validate_features(profile_view4, True)

In [ ]:
visualization = NotebookProfileVisualizer()
visualization.constraints_report(const, cell_height=300)

In [ ]:
# check all constraints for passing:
constraints_valid = const.validate()
print(constraints_valid)

In [ ]:
profile_view4.to_pandas()

Leran more about performing data validation with whylogs
- [Data Validation with Metric Constraints](https://github.com/whylabs/whylogs/blob/mainline/python/examples/advanced/Metric_Constraints.ipynb)
